## This is a notebook where the supporting plots in the main notebook can be reproduced. These run on either the statevector or simulated real backends, so no need to connect to IBMQ.

### 1 Imports

In [ ]:
%pip install quantum-isl
%pip install mpl_interactions
%pip install ipympl
%pip install qiskit==0.34.0

In [ ]:
# Importing standard Qiskit modules
from qiskit import QuantumCircuit, QuantumRegister, IBMQ, execute, transpile, Aer
from qiskit.tools.monitor import job_monitor
from qiskit.providers.aer import AerSimulator
from qiskit.test.mock import FakeJakarta
from qiskit.result import Result
from qiskit.ignis.mitigation.measurement import *

# Import state tomography modules
from qiskit.ignis.verification.tomography import state_tomography_circuits, StateTomographyFitter
from qiskit.quantum_info import state_fidelity

# Import python scripts
import analysis_utils 
import experiment_utils 
import misc_utils 
import file_utils

# suppress warnings
import warnings
warnings.filterwarnings('ignore')

In [ ]:
statevector = Aer.get_backend('statevector_simulator')
fake_backend = AerSimulator.from_backend(FakeJakarta())

backends = {'statevector' : statevector,
            'fake_jakarta' : fake_backend}


### Compute exact time evolution of |110> state under Heisenberg model.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import mpl_interactions.ipyplot as iplt
plt.rcParams.update({'font.size': 16})  # enlarge matplotlib fonts

# Import qubit states Zero (|0>) and One (|1>), and Pauli operators (X, Y, Z)
from qiskit.opflow import Zero, One, I, X, Y, Z

# Suppress warnings
import warnings
warnings.filterwarnings('ignore')


# Returns the matrix representation of the XXX Heisenberg model for 3 spin-1/2 particles in a line
def H_heis3():
    # Interactions (I is the identity matrix; X, Y, and Z are Pauli matricies; ^ is a tensor product)
    XXs = (I^X^X) + (X^X^I)
    YYs = (I^Y^Y) + (Y^Y^I)
    ZZs = (I^Z^Z) + (Z^Z^I)
    
    # Sum interactions
    H = XXs + YYs + ZZs
    
    # Return Hamiltonian
    return H


# Returns the matrix representation of U_heis3(t) for a given time t assuming an XXX Heisenberg Hamiltonian for 3 spins-1/2 particles in a line
def U_heis3(t):
    # Compute XXX Hamiltonian for 3 spins in a line
    H = H_heis3()
    
    # Return the exponential of -i multipled by time t multipled by the 3 spin XXX Heisenberg Hamilonian 
    return (t * H).exp_i()

# Define array of time points
ts = np.linspace(0, np.pi, 100)

# Define initial state |110>
initial_state = One^One^Zero

# Compute probability of remaining in |110> state over the array of time points
 # ~initial_state gives the bra of the initial state (<110|)
 # @ is short hand for matrix multiplication
 # U_heis3(t) is the unitary time evolution at time t
 # t needs to be wrapped with float(t) to avoid a bug
 # (...).eval() returns the inner product <110|U_heis3(t)|110>
 #  np.abs(...)**2 is the modulus squared of the innner product which is the expectation value, or probability, of remaining in |110>
probs_110 = [np.abs((~initial_state @ U_heis3(float(t)) @ initial_state).eval())**2 for t in ts]

### 3-1 Determining Minimum Number of Necessary Trotter Steps 
Now that we understand how ISL can provide shallower approximate equivalents of quantum circuits, we move on to the problem at hand. If we are to simulate the evolution of the XXX model on real quantum hardware there will be two sources of error: from the device and from our Suzuki-Trotter decomposition. In this section, we'd first like to eliminate the latter, so that we can focus purely on error mitigation of the device. Since the Trotter error grows as the size of the discrestised time slice $\delta t$, we aim to find the number of time slices required for it to be negligible.

Here we define negligible Trotter error as the minimum number of Trotter steps required to reproduce the original |110> evolution graph within visual convergence. This is an alternative to other methods of estimating Trotter error.

In [ ]:
''' 
    Looking at the problem in more generality, what is the minimum Trotter step required to reproduce whole time 
    evolution of probability within some margin of error
    
    
    Produce convergence of the default Trotter solution (or any Trotterisation, make it plug & play) to the exact
    probability evolution. This is a bit redundant for the code but is minimum necessary for us to justify the 
    generality of ISL
    
    min_ts : Minimum trotter step
''' 
target_time = np.pi
init_trotter_steps = 4
max_trotter_steps = 37
num_qubits = 7
backend = statevector
shots = 8192
reps = 8

yss_min_ts = []
xss_min_ts = []
markers_min_ts = []
labels_min_ts = []
alphas_min_ts = []

for trot_step in range(init_trotter_steps, max_trotter_steps): 
    print('{} out of {}'.format(trot_step, max_trotter_steps - 1))
    state_probs_at_each_trotter_step = []
    
    # returns jobs for each incrementally growing Trotter circuit 
    # e.g. -T(pi/20)-, -T(pi/20)-T(pi/20),....
    # Each of these approximates the evolution of the giving Hamiltonian at increasing time steps
    # Having a larger 'max_trotter_steps' will mean more of the circuits, each parameterised by a smaller time slice.
    jobs_for_each_trotter_step = experiment_utils.simulate_heisenberg_xxx_over_time(target_time, trot_step, num_qubits, backend, shots, reps, False)
    
    for jobs in jobs_for_each_trotter_step: # the results of executing the trotter circuit for each time step.
        # get state probabilities from counts and average over multiple reps
        state_probs = analysis_utils.state_probabilities_from_circuit_multi_reps(jobs)
        state_probs_at_each_trotter_step.append(state_probs)
                    
    target_state = '0101000'
    # extract probability of being in target state for each timestep
    yss_min_ts.append([state_probs[target_state] if target_state in state_probs else 0 for state_probs in state_probs_at_each_trotter_step])
    xss_min_ts.append([i*(target_time / trot_step) for i in range(0, trot_step + 1)])
    labels_min_ts.append(trot_step)
    markers_min_ts.append('g+')
    alphas_min_ts.append(0.5)
    
yss_min_ts.append(probs_110)
xss_min_ts.append(ts)
labels_min_ts.append('Exact')
markers_min_ts.append('-')
alphas_min_ts.append(1)
    

In [ ]:
%matplotlib ipympl

# define callback functions for slider
def f_x(trotter_step):
    return xss_min_ts[trotter_step - init_trotter_steps]

def f_y(x, trotter_step):
    return yss_min_ts[trotter_step - init_trotter_steps]


xlabel = 'Time'
ylabel = 'Probability of 110'
legend = False

plt.plot(xss_min_ts[-1], yss_min_ts[-1], markers_min_ts[-1], alpha=alphas_min_ts[-1], label='exact')

all_trotter_steps = list(range(init_trotter_steps, max_trotter_steps))
controls = iplt.plot(f_x, f_y, 'ko', trotter_step=all_trotter_steps, clip_on=False, label='Approx')

plt.legend()
plt.xlim(0,np.pi)
plt.xlabel(xlabel)
_ = plt.ylabel(ylabel)


Adjusting the slider we can see how increasing the number of Trotter steps changes the solution in two ways. Firstly, increased Trotter steps leads to smaller time slices, producing more discrete points across the evolution. Most importantly however, the smaller time slices leads to reduced error from the decomposition, such that we observe convergence with the analytic solution as the slider is increased. Overall, we see that 35 Trotter steps is sufficient for reproducing the probability evolution from 0 to pi to a level where the error will be insignificant compared to the device noise.

In [ ]:
min_trotter_steps_visual_convergence = 35

Now we have an idea of the number of Trotter steps we need to approximate time evolution within our error threshold. We will recompile the circuit corresponding to this number of steps with ISL and see how close the result is on the noise free backend.

### 3-2 ISL Reproduces Full Probability Evolution

Note, that, in the interest of time, the circuit recompilation here uses cached results if a previously recompiled circuit is passed in. Producing this plot involves doing the same recompilations as those in the full probability evolution plot in the 'run it yourself' section at the end of the main notebook. In this section these recompilations are done from scratch / without using cached results. 

In [ ]:
'''
    Produce comparison between probability evolution of 110 state with time for an exact solution, the default 
    Trotter step and the default Trotter step recompiled with ISL.
'''
target_time = np.pi
trotter_steps = min_trotter_steps_visual_convergence
num_qubits = 7
backend = statevector
shots = 8192
reps = 8
sufficient_cost = 1e-3
target_state = '0101000'


fs_full_pe = [file_utils.get_partial_trotter_circuit_state_probs_filename('0','pi', trotter_steps, i, backend.name(), shots, reps) for i in range(trotter_steps+1)]

# load these results from section 3-1
try:
    state_probs_at_each_trotter_step = []
    for f in fs_full_pe:
        probs = np.load(f + '.npy', allow_pickle=True).tolist()
        state_probs_at_each_trotter_step.append(probs)
except FileNotFoundError: 
    print('couldnt find file')

# note the isl flag is now True in the simulate_heisenberg_... function
# construct trotter simulation circuits, recompile, then execute for increasing trotter steps up to a max of trotter_steps
jobs_for_each_trotter_step_isl = experiment_utils.simulate_heisenberg_xxx_over_time(target_time, trotter_steps, num_qubits, backend, shots, reps, True, sufficient_cost)
state_probs_at_each_trotter_step_isl = []

for jobs in jobs_for_each_trotter_step_isl:
    state_probs = analysis_utils.state_probabilities_from_circuit_multi_reps(jobs)
    state_probs_at_each_trotter_step_isl.append(state_probs)
        
ys_full_pe = [state_probs[target_state] if target_state in state_probs else 0 for state_probs in state_probs_at_each_trotter_step]
ys_isl_full_pe = [state_probs[target_state] if target_state in state_probs else 0 for state_probs in state_probs_at_each_trotter_step_isl]
xss_full_pe = [ts] + [[i*(target_time/trotter_steps) for i in range(0, trotter_steps+1)] for _ in range(2)]
yss_full_pe = [probs_110, ys_full_pe, ys_isl_full_pe]

labels_full_pe = ['exact','default trotter','isl']
markers_full_pe = ['-','o', 'x']
title_full_pe = 'Max Trotter steps: {}, suff cost: {}, shots: {}. reps: {}'.format(trotter_steps, sufficient_cost, shots, reps)


In [ ]:
# reset the matplotlib backend
%matplotlib ipympl

misc_utils.xys_plot(plt, xss_full_pe, yss_full_pe, 'Time', 'Probability of 110',labels_full_pe, markers_full_pe, title_full_pe, alphas=[1 for i in range(len(xss_full_pe))])


Here we see that for a trial sufficient cost of 0.001 (99.9% overlap between the original and recompiled states), our recompiled solution is able to reproduce the observed population over the full evolution. Most importantly, it is able to do so with significantly shallower circuits than the direct Trotterised implementation. Here the Trotterised evolution circuits contain 10 CNOT gates per Trotter step (transpiling with optimisation level 3). By comparison, ISL finds an approximately equivalent circuit to within 99.9% overlap with on average 3 CNOT gates, for **any** number of Trotter steps. This difference is most notable when obtaining the population at the final time $t=\pi$, where the direct Trotterised implementation requires 10*35=350 CNOT gates.

Now let's look at how both methods perform on the fake Jakarta backend.

### 3-3 Real Device Performance

In [ ]:
'''
    Produce comparison between probability evolution of 110 state with time for an exact solution, the default 
    Trotter step and the default Trotter step recompiled with ISL.
'''
backend = fake_backend
shots = 8192
reps = 8

jobs_for_each_trotter_step = experiment_utils.simulate_heisenberg_xxx_over_time(target_time, trotter_steps, num_qubits, backend, shots, reps, False)
state_probs_at_each_trotter_step = []

for jobs in jobs_for_each_trotter_step:
    state_probs = analysis_utils.state_probabilities_from_circuit_multi_reps(jobs)
    state_probs_at_each_trotter_step.append(state_probs)

ys_rdp = [state_probs[target_state] if target_state in state_probs else 0 for state_probs in state_probs_at_each_trotter_step]



In [ ]:
ys_isl_for_each_suff_cost_rdp = []
sufficient_costs = [1e-1, 1e-2, 1e-3, 1e-4, 1e-5, 1e-6]

for sufficient_cost in sufficient_costs:
    jobs_for_each_trotter_step_isl = experiment_utils.simulate_heisenberg_xxx_over_time(target_time, trotter_steps, num_qubits, backend, shots, reps, True, sufficient_cost)
    state_probs_at_each_trotter_step_isl = []

    for jobs in jobs_for_each_trotter_step_isl:
        state_probs = analysis_utils.state_probabilities_from_circuit_multi_reps(jobs)
        state_probs_at_each_trotter_step_isl.append(state_probs)
            
    ys_isl_for_each_suff_cost_rdp.append([state_probs[target_state] if target_state in state_probs else 0 for state_probs in state_probs_at_each_trotter_step_isl])


In [ ]:
# reset the matplotlib backend
%matplotlib ipympl

def f_y(x, sufficient_cost):
    title = 'Max Trotter steps: {}, suff cost: {}, shots: {}. reps: {}'.format(trotter_steps, sufficient_cost, shots, reps)
    plt.title(title)
    return ys_isl_for_each_suff_cost_rdp[sufficient_costs.index(sufficient_cost)]

xs_rdp = [i*(target_time/trotter_steps) for i in range(0, trotter_steps+1)] 

xlabel = 'Time'
ylabel = 'Probability of 110'

plt.plot(ts, probs_110, '-', label='exact', clip_on=False)
plt.plot(xs_rdp, ys_rdp, 'o', label='default trotter', clip_on=False)

controls = iplt.plot(xs_rdp, f_y, 'x', sufficient_cost=sufficient_costs, clip_on=False, label='isl', slider_formats={"sufficient_cost": "{:.1e}"})

plt.legend()
plt.xlim(0,np.pi)
plt.xlabel(xlabel)
_ = plt.ylabel(ylabel)
